# Module 5 Homework - Let's Get Lunch

Suppose we are looking for a good place for lunch on our next trip to La Crosse. Let's collect some data on nearby restaurants.

## <font color="red"> Problem 1 </font>

Goto yelp.com and perform a search with the following parameters

* **Find** Restaurants
* **Near** La Crosse, WI

**Tasks**
1. Copy the resulting web address below and determine the how the     specified search terms related to the resulting address
2. Use requests and Beautiful Soap to download the content of the front page.

In [24]:
from composable import pipeable
from composable.strict import map, filter
from composablesoup import find, find_all, get_text, has_attr
from composablesoup.soup import find_parent, parents, children, find_previous_sibling, find_previous_siblings, find_next_sibling, find_next_siblings, find_previous_sibling
from composable.sequence import to_list, head
from composable.string import strip
from composable.string import replace
from composable import from_toolz as tlz

In [2]:
import requests
from bs4 import BeautifulSoup
s = requests.Session()
r = s.get('https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI')
yelp = BeautifulSoup(r.content, "html.parser")

## <font color="red"> Problem 2 </font>

We want to grab the restaurant's name.

1. Use Inspect Element to determine the tags/classes for each of the elements listed above.  
2. Note that all the business information is contained in a`div` that contains a class similar to  `<div class=" ... businessName__09f24__3Wql2 ...">`.  You will need to use a regular expression to match the `businessName` in the class (see lecture 5.3).
3. Write expressions/functions to pull out the name of each restaurant.  
    * Note: The business name is in an unnamed tag, you will need to navigate to the using searches and/or relationship
 
**Confirm that there is an extra restaurant in the list (e.g. 11-12 instead of 10). This is due to an advertisement/sponsered links, which we want to ignore.**

In [65]:
import re
businessName = re.compile('businessName')
numbers = re.compile('\d{1-10}.')
clean_space = pipeable(lambda l: [line[line.find('.') + 1:] for line in l])
remove_first = pipeable(lambda l: l[1:])
(yelp
 >> find_all('div', class_ = businessName)
 >> map(get_text)
 >> map(replace('\xa0',''))
 >> clean_space
 >> remove_first
)

['Restore Public House',
 'The Waterfront Restaurant & Tavern',
 'Buzzard Billy’s',
 'Lovechild Restaurant',
 'Howie’s on La Crosse',
 'The Charmant',
 'The Freighthouse Restaurant',
 'The Crow',
 'Schuby’s Neighborhood Butcher',
 'Piggy’s Restaurant']

In [66]:
get_name = pipeable(lambda soup: soup
                    >> find_all('div', class_ = businessName)
                    >> map(get_text)
                    >> map(replace('\xa0',''))
                    >> clean_space
                    >> remove_first
                    )
yelp >> get_name


['Restore Public House',
 'The Waterfront Restaurant & Tavern',
 'Buzzard Billy’s',
 'Lovechild Restaurant',
 'Howie’s on La Crosse',
 'The Charmant',
 'The Freighthouse Restaurant',
 'The Crow',
 'Schuby’s Neighborhood Butcher',
 'Piggy’s Restaurant']

## <font color="red"> Problem 3 </font>

Since we picked up extra information, we will need to be clever about identifying the information block for each restaurant.  Note that all of the actual search results (but not sponsered links) start with the ranking (e.g. `11.`).  Use the following steps to get a list that contains the information for each restaurant other than the adds.

1. Start by finding the ranking of the restaurant (1., 2., etc.). **Hint:** You will need to use regular expression to match the text of the tag (see lecture 5.3).
2. Now search for a parent of the above tags that surrounds all of the restaurant information.  You will want to use the `find_parent` method on each of the tags from **1.**.  **Hint:** Look through each of the `div` tags that contain the ranking, looking for a meaningful tag name to match with a regular expression.

The resulting list will be the starting point for gathering all of the information.

In [73]:
get_first_index = pipeable(lambda l:[line[0: line.find('.')] for line in l])
(yelp
 >> find_all('div', class_ = businessName)
 >> map(get_text)
 >> remove_first
 >> get_first_index
)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [74]:
get_rank = pipeable(lambda soup: soup
                   >> find_all('div', class_ = businessName)
                   >> map(get_text)
                   >> remove_first
                   >> get_first_index
                   )
yelp >> get_rank

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

## <font color="red"> Problem 4 </font>

Write expressions/functions to gather each of the following pieces of information for each of the restaurants.

#### Restaurant Name

In [75]:
get_name = pipeable(lambda soup: soup
                    >> find_all('div', class_ = businessName)
                    >> map(get_text)
                    >> map(replace('\xa0',''))
                    >> clean_space
                    >> remove_first
                    )
yelp >> get_name

['Restore Public House',
 'The Waterfront Restaurant & Tavern',
 'Buzzard Billy’s',
 'Lovechild Restaurant',
 'Howie’s on La Crosse',
 'The Charmant',
 'The Freighthouse Restaurant',
 'The Crow',
 'Schuby’s Neighborhood Butcher',
 'Piggy’s Restaurant']

#### Rating

In [ ]:
get_rank = pipeable(lambda soup: soup
                   >> find_all('div', class_ = businessName)
                   >> map(get_text)
                   >> remove_first
                   >> get_first_index
                   )
yelp >> get_rank

#### Address

In [86]:
raw = re.compile('raw')
get_addresses_lines = pipeable(lambda l: [line for line in l if line[0].isnumeric()])
(yelp
 >> find_all('span', class_= raw)
 >> map(get_text)
 >> get_addresses_lines
 >> remove_first
)

['1810 State St',
 '328 Front St S',
 '222 Pearl St',
 '300 3rd St S',
 '1128 La Crosse St',
 '101 State St',
 '107 Vine St',
 '100 3rd St S',
 '321 State St',
 '501 Front St S']

In [88]:
get_address = pipeable(lambda soup: soup
              >> find_all('span', class_= raw)
              >> map(get_text)
              >> get_addresses_lines
              >> remove_first
              )
yelp >> get_address

['1810 State St',
 '328 Front St S',
 '222 Pearl St',
 '300 3rd St S',
 '1128 La Crosse St',
 '101 State St',
 '107 Vine St',
 '100 3rd St S',
 '321 State St',
 '501 Front St S']

#### Review Count

In [92]:
review_count = re.compile('reviewCount')
(yelp
 >> find_all('span', class_= review_count)
 >> map(get_text)
 >> remove_first
)

['17', '226', '275', '109', '57', '149', '125', '154', '17', '131']

In [93]:
get_review_count = pipeable(lambda soup: soup
                           >> find_all('span', class_= review_count)
                           >> map(get_text)
                           >> remove_first
                           )

yelp >> get_review_count

['17', '226', '275', '109', '57', '149', '125', '154', '17', '131']

#### Category

**Examples:** `['American (New)', 'Seafood', 'Steakhouses']` becomes `'American (New);Seafood;Steakhouses'`

In [102]:
link = re.compile('link-color')
(yelp
 >> find_all('a', class_ = link)
 >> map(get_text) 
)

['Yelp',
 'See all',
 'See all',
 'Recommended',
 '',
 'Pappi’s Taqueria y Mas',
 'Mexican',
 'Food Trucks',
 'more',
 '',
 'Restore Public House',
 'American (Traditional)',
 'more',
 '',
 'The Waterfront Restaurant & Tavern',
 'American (New)',
 'Seafood',
 'Steakhouses',
 'more',
 '',
 'Buzzard Billy’s',
 'American (Traditional)',
 'Cajun/Creole',
 'more',
 '',
 'Lovechild Restaurant',
 'American (New)',
 'more',
 '',
 'Howie’s on La Crosse',
 'American (New)',
 'Pubs',
 'more',
 '',
 'The Charmant',
 'French',
 'Cocktail Bars',
 'more',
 '',
 'The Freighthouse Restaurant',
 'Seafood',
 'Steakhouses',
 'Desserts',
 'more',
 '',
 'The Crow',
 'American (Traditional)',
 'Gastropubs',
 'more',
 '',
 'Schuby’s Neighborhood Butcher',
 'Butcher',
 'Delis',
 'Caterers',
 'more',
 '',
 'Piggy’s Restaurant',
 'Steakhouses',
 'Seafood',
 'Sandwiches',
 'more',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '',
 'Help us improve.',
 'Virtual Cooking Classes',
 'Bbq Restaurants',
 'Dinner Res

## <font color="red">  Problem 4 </font>

Package all of the expressions in a function that takes a url as input and returns the table of information.  Use a `def` statement and put the above helper functions in the body of the main function.  Test this function on the front page of the search.

## <font color="red">  Problem 5 </font>

Now perform a linear search to grab all of the information on restaurants in La Crosse.  You will need to

1. Inspect the url for the first, second, etc. pages to determine the pattern.
2. Create a list of urls for all search results.
3. Get the info from all pages and use your function from the last problem to parse the data into a list.
4. Write the results to a csv file. **Hint:** Use `'a'` (append) instead of `'w'` write on all after the first/

## <font color="red">  Bonus Problem </font>

See if you can also grab the latitude and longitude of each result.